In [ ]:
from tqdm.notebook import tqdm

In [ ]:
import json
# import pandas as pd
# import polars as pl
import unicodedata

import polars as pl
import torch
# import torch.nn as nn
# import torch.optim as optim
from torch.utils.data import Dataset#, DataLoader
import numpy as np
from sentence_transformers import SentenceTransformer


In [ ]:
df_train = pl.read_csv("data/imported/training.csv")
df_test = pl.read_csv("data/imported/testing.csv")
df_validation = pl.read_csv("data/imported/val.csv")

train_texts = df_train["text"].to_list()
train_labels = df_train["score"].to_list()

test_texts = df_test["text"].to_list()
test_labels = df_test["score"].to_list()

val_texts = df_validation["text"].to_list()
val_labels = df_validation["score"].to_list()


In [ ]:
# print(min(df_train['score'].unique()))
# print(max(df_train['score'].unique()))
# # 
# # # 0 to 60, all integers

In [ ]:
# embed text
# model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "intfloat/multilingual-e5-large-instruct"
print("loading embedder")
embedder = SentenceTransformer(model_name, device="cuda")


In [ ]:
print("embedding train dataset")
train_embeddings = embedder.encode(train_texts, batch_size=16, convert_to_numpy=True, show_progress_bar=True)

In [ ]:
print("embedding test dataset")
test_embeddings = embedder.encode(test_texts, batch_size=16, convert_to_numpy=True,show_progress_bar=True)

In [ ]:
print("embedding val dataset")
val_embeddings = embedder.encode(val_texts, batch_size=16, convert_to_numpy=True,show_progress_bar=True)

In [ ]:
class EssayScoreDataset(Dataset):
    def __init__(self, X, y):
        """
        Args:
            X (list or array-like): Feature data.
            y (list or array-like): Essay scores (integers from 0 to 60).
        """
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)  # Input features as float32
        y = torch.tensor(self.y[idx], dtype=torch.float16)  # Scores as float16
        return x, y


In [ ]:
train_dataset = EssayScoreDataset(train_embeddings, train_labels)
test_dataset = EssayScoreDataset(test_embeddings, test_labels)
val_dataset = EssayScoreDataset(val_embeddings, val_labels)


In [ ]:
# torch.save(train_dataset, 'data/embeddings/train_dataset00.pth')
# torch.save(test_dataset, 'data/embeddings/test_dataset00.pth')
# torch.save(val_dataset, 'data/embeddings/val_dataset00.pth')
# 


In [ ]:
torch.save(train_dataset, 'data/embeddings/train_dataset01.pth')
torch.save(test_dataset, 'data/embeddings/test_dataset01.pth')
torch.save(val_dataset, 'data/embeddings/val_dataset01.pth')



# fit a model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


In [2]:
class EssayScoreDataset(Dataset):
    def __init__(self, X, y):
        """
        Args:
            X (list or array-like): Feature data.
            y (list or array-like): Essay scores (integers from 0 to 60).
        """
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = torch.tensor(self.X[idx], dtype=torch.float32)  # Input features as float32
        y = torch.tensor(self.y[idx], dtype=torch.float16)  # Scores as float16
        return x, y


In [3]:
train_dataset = torch.load('data/embeddings/train_dataset01.pth')
test_dataset = torch.load('data/embeddings/train_dataset01.pth')
val_dataset = torch.load('data/embeddings/train_dataset01.pth')



/tmp/ipykernel_18691/3384220254.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('data/embeddings/train_dataset01.pth')
/tmp/ipykernel_18691/33

In [4]:
# Wrap datasets in DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [5]:
# Define a simple feedforward neural network
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
train_dataset[0][0].shape

In [6]:
# Assuming input size and output size are known (adjust as needed)
input_size = 1024  # Change based on your dataset
hidden_size = 64
output_size = 1  # For MSE loss, often 1 output

# Initialize model, loss function, and optimizer
model = SimpleMLP(input_size, hidden_size, output_size).to('cuda')
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
# def train_epoch(model, loader, criterion, optimizer):
#     model.train()
#     running_loss = 0.0
#     for inputs, targets in loader:
#         inputs, targets = inputs.to('cuda'), targets.to('cuda').unsqueeze(1)  # Ensure targets have shape [batch_size, 1]
# 
#         # Forward pass
#         outputs = model(inputs)
#         loss = criterion(outputs, targets)
# 
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
# 
#         running_loss += loss.item()
# 
#     return running_loss / len(loader)


In [ ]:
# def evaluate_model(model, loader, criterion):
#     model.eval()
#     running_loss = 0.0
#     with torch.no_grad():
#         for inputs, targets in loader:
#             inputs, targets = inputs.to('cuda'), targets.to('cuda').unsqueeze(1)  # Ensure targets have shape [batch_size, 1]
# 
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
# 
#             running_loss += loss.item()
# 
#     return running_loss / len(loader)


In [ ]:
# # Main training and evaluation
# num_epochs = 10
# for epoch in range(num_epochs):
#     train_loss = train_epoch(model, train_loader, criterion, optimizer)
#     val_loss = evaluate_model(model, val_loader, criterion)
# 
#     print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

In [ ]:
# # Evaluate on test set
# test_loss = evaluate_model(model, test_loader, criterion)
# print(f"Test Loss: {test_loss:.4f}")

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Load datasets
train_dataset = torch.load('data/embeddings/train_dataset01.pth')
test_dataset = torch.load('data/embeddings/test_dataset01.pth')
val_dataset = torch.load('data/embeddings/val_dataset01.pth')

# Wrap datasets in DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define a simple feedforward neural network
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Assuming input size and output size are known (adjust as needed)
input_size = 1024  # Change based on your dataset
hidden_size = 64
output_size = 1  # For MSE loss, often 1 output

# Initialize model, loss function, and optimizer
model = SimpleMLP(input_size, hidden_size, output_size).to('cuda').to(torch.float32)  # Ensure model uses float32
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3)

# Training loop
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    for inputs, targets in loader:
        inputs, targets = inputs.to('cuda').to(torch.float32), targets.to('cuda').to(torch.float32).unsqueeze(1)  # Ensure inputs and targets use float32

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)

# Evaluation loop
def evaluate_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to('cuda').to(torch.float32), targets.to('cuda').to(torch.float32).unsqueeze(1)  # Ensure inputs and targets use float32

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            running_loss += loss.item()

    return running_loss / len(loader)

# Main training and evaluation
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    val_loss = evaluate_model(model, val_loader, criterion)

    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# Evaluate on test set
test_loss = evaluate_model(model, test_loader, criterion)
print(f"Test Loss: {test_loss:.4f}")

/tmp/ipykernel_18691/4267202092.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('data/embeddings/train_dataset01.pth')
/tmp/ipykernel_18691/42

Epoch 1/10, Train Loss: 75.3640, Validation Loss: 68.3049
Epoch 2/10, Train Loss: 32.9435, Validation Loss: 21.6173
Epoch 3/10, Train Loss: 12.2645, Validation Loss: 10.1181
Epoch 4/10, Train Loss: 8.2918, Validation Loss: 8.2402
Epoch 5/10, Train Loss: 7.4300, Validation Loss: 8.4053
Epoch 6/10, Train Loss: 6.9486, Validation Loss: 7.4242
Epoch 7/10, Train Loss: 6.4938, Validation Loss: 7.3517
Epoch 8/10, Train Loss: 6.2903, Validation Loss: 6.8462
Epoch 9/10, Train Loss: 5.9002, Validation Loss: 6.7228
Epoch 10/10, Train Loss: 5.6832, Validation Loss: 6.5828
Test Loss: 6.0776


In [8]:
evaluate_model(model, val_loader, criterion)

6.58279725164175

In [ ]:
from sklearn.metrics import mean_squared_error

# TODO
## just guess the mean/median
## just ask chatgpt
## just use small embed+NN
## just use big embed+NN
## just ask llama
## llama finetune
## llama as embedder 